In [ ]:
pip install gradio

In [16]:
import gradio as gr
import pandas as pd
import json 
from FactureModel import Facture
import constants as const
from gradio import Interface, Image, Checkbox, Textbox
from functions import readDataSet, imageObjCreation, readImgOCR, pydanticParser, instructionsFormat, LLMModelCall, query_pinecone

def process_invoice(image_path, do_easyocr):
    # Step 1: Process the image to extract text using OCR
    with open(image_path, "rb") as f:
        image_bytes = f.read()
    image_obj = imageObjCreation(image_bytes)
    ocr_text = readImgOCR(image_bytes, image_obj, do_easyocr)

    # Step 2: Setup Pydantic parser and format instructions
    parser = pydanticParser()
    system_instructions, prompt = instructionsFormat(parser, ocr_text)

    # Step 3: Call the LLM model to extract invoice data
    extracted_data = LLMModelCall(prompt, system_instructions)
    facture_data = parser.parse(extracted_data)
   
    date = facture_data.date
    monto = facture_data.monto
    facture_number = facture_data.facture_number

    
    return date,monto,facture_number

def query_database(query_vector_text):
    try:
        query_vector = json.loads(query_vector_text)
        db_results = query_pinecone(query_vector)
    except json.JSONDecodeError:
        db_results = "Invalid query vector format. Please enter a valid JSON array."
    except Exception as e:
        db_results = str(e)

    return db_results

def invoice_interface():
    invoice_app = gr.Interface(
        fn=process_invoice,
        inputs=[
            gr.Image(type="filepath", label="Upload Invoice Image"),
            gr.Checkbox(label="Use EasyOCR"),
        ],
        outputs=[
            gr.Textbox(label="Date"),
            gr.Textbox(label="Monto"),
            gr.Textbox(label="Facture Number")
        ],
        title="Invoice Data Extractor",
        description="Upload an invoice image and extract data from it using OCR and LLM"
    )
    invoice_app.launch()

def pinecone_query_interface():
    query_app = gr.Interface(
        fn=query_database,
        inputs=[
            gr.Textbox(label="Query Vector", placeholder="Enter query vector as a list of numbers")
        ],
        outputs=[
            gr.Textbox(label="Pinecone Results")
        ],
        title="Pinecone Database Query",
        description="Enter a query vector to search the Pinecone database."
    )
    query_app.launch()

def main_menu(choice):
    if choice == "Extract Invoice Data":
        invoice_interface()
    elif choice == "Query Pinecone Database":
        pinecone_query_interface()

menu_choices = ["Extract Invoice Data", "Query Pinecone Database"]
menu = gr.Interface(
    fn=main_menu,
    inputs=gr.Dropdown(menu_choices, label="Choose an option"),
    outputs=None,
    title="Main Menu",
    description="Select an option to proceed"
)

# Launch the main menu
menu.launch()

Running on local URL:  http://127.0.0.1:7872

To create a public link, set `share=True` in `launch()`.


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "s:\anaconda3\Lib\site-packages\uvicorn\protocols\http\httptools_impl.py", line 399, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "s:\anaconda3\Lib\site-packages\uvicorn\middleware\proxy_headers.py", line 70, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "s:\anaconda3\Lib\site-packages\fastapi\applications.py", line 1054, in __call__
    await super().__call__(scope, receive, send)
  File "s:\anaconda3\Lib\site-packages\starlette\applications.py", line 123, in __call__
    await self.middleware_stack(scope, receive, send)
  File "s:\anaconda3\Lib\site-packages\starlette\middleware\errors.py", line 186, in __call__
    raise exc
  File "s:\anaconda3\Lib\site-packages\starlette\middleware\errors.py", line 164, in __call__
    await self.app(scope,

Texto extraído con OCR:
~i2i J;_il ~5001+ { ~c ! Auro-enlrepraneur Date 28/03/2023 Facture numéro 000 001 Client Smart Call Distribution Adresse 3, Angle Rues Alfred de Musset et Abdelkader El Mazini ler Etage, Quartier Gautier, Casablanca Dësignátíön 'Quantité Prixunitaire Total RD.V 30 120,00 DH 3600.00 DH AVANCE SUR REMUNERATION 500,00 500,00 Montant en dirhams Total Net à payer 3100,00 (Hors champ de la TVA') ARRETE LA PRESENTE FACTURE A LA SOMME DE #TROIS MILLE CENT DIRHAMS # Fifltukeh S8yY 'Art 89 II = 1' c, Code Général des Impôts. ICE: Auto Entrepreneur Mouhcine FTOUHI CNIE A 251909 Adresse 3, Imm 97, GH 23, Lotissement Firdaous, Oulfa, Casblanca ICE (N' d'inscription au registre national de lauto-entrepreneur) 003159152000040 IF : 52682016 Taxe professionnelle N' 25909366 TEL 0696579734 Mail Mouhcine22f@gmail com Mouncho Conaz Auto-Entr OoCw 00315
